In [ ]:
# | default_exp climate_utils

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import is_close

In [ ]:
# | export
import warnings
import operator
import collections
import numpy as np
from math import pi
from typing import Dict
from numpy import exp, cos, sin, arccos, arctan
# from pysureau.parameter_validators import ClimateDataValidator

In [ ]:
# | export
def day_length(
    latitude: float,  # numeric value specifying the geographic latitude (in decimal degrees) of the location of interest
    day_of_year: int,  # numeric (usually integer) value or vector specifying the Julian day (day of the year), for which calculations should be done.
    no_times_as_na: bool = False,  # parameter to determine whether for days without sunrise or sunset, na should be returned for Sunset and Sunrise. If left at FALSE, the function returns -99 and 99 for sunrise and sunset or polar nights and polar days, respectively
) -> Dict:  # Dictionary with three elements Sunrise, Sunset and Daylength. For days without sunrise (polar nights),sunset and sunrise become -99 and the daylength 0. For days without sunset, sunset and sunrise are 99 and daylength 24.
    "Original function from chillR R package. This function computes sunrise time, sunset time and daylength for a particular location and day of the year (Julian day). This is done using equations by Spencer (1971) and Almorox et al. (2005)."

    warnings.warn('Double check day_length function works for Australia')

    # Assert parameters ---------------------------------------------------------
    # Latitude
    assert (
        isinstance(latitude, float) | isinstance(latitude, int)
        and 95 >= latitude >= -95
    ), (
        'Provide latitude as coordinates points bewteen -90 and 90 i.e. latitude = 41.40338'
    )

    # Day of year
    # Using np.testing instead of assert because parameters can be np.arrays OR
    # single values (i.e. 1). assert only works when params are always one type
    # Solution from:
    # https://stackoverflow.com/questions/45987962/why-arent-there-numpy-testing-assert-array-greater-assert-array-less-equal-as

    np.testing.assert_array_compare(
        operator.__gt__,
        np.array(day_of_year),
        0,
        err_msg='\nday_of_year must be must be a integer value between 1-366\n',
    )

    np.testing.assert_array_less(
        np.array(day_of_year),
        367,
        err_msg='\nError: day_of_year must be must be a integer value between 1-366\n',
    )

    # Define constants ----------------------------------------------------------
    gamma = 2 * (pi / 365) * ((day_of_year) - 1)
    delta = (180 / pi) * (
        0.006918
        - 0.399912 * cos(gamma)
        + 0.070257 * sin(gamma)
        - 0.006758 * cos(2 * gamma)
        + 0.000907 * sin(2 * (gamma))
        - 0.002697 * cos(3 * (gamma))
        + 0.00148 * sin(3 * (gamma))
    )

    cos_wo = (
        sin((-0.8333 / 360) * 2 * pi)
        - sin(latitude / 360 * 2 * pi) * sin((delta / 360) * 2 * pi)
    ) / (cos((latitude / 360) * 2 * pi) * cos((delta / 360) * 2 * pi))

    # Step implemented in case day_of_year is a single value i.e.
    # day_of_year = 80
    if isinstance(cos_wo, float):
        # Transform float into array
        cos_wo = np.array([cos_wo])

    # Calculate sunrise and sunset ----------------------------------------------

    # Get the position of values between -1 and 1
    # Note only day_of_year values from 77 to 80 and will return values
    # between -1 and 1

    normal_days = np.where((cos_wo >= -1) & (cos_wo <= 1))

    # Initialize a numpy arrays

    sunrise = np.full(len(cos_wo), -99, dtype=float)
    sunset = np.full(len(cos_wo), -99, dtype=float)

    # Add normal days to sunrise and sunset arrays
    sunrise[normal_days] = 12 - arccos(cos_wo[normal_days]) / (15 / 360 * 2 * pi)
    sunset[normal_days] = 12 + arccos(cos_wo[normal_days]) / (15 / 360 * 2 * pi)

    # Calculate day length
    day_length = sunset - sunrise

    day_length[np.where(cos_wo > 1)] = 0
    day_length[np.where(cos_wo < (-1))] = 24

    # Replace values with 99
    sunrise[np.where(day_length == 24)] = 99
    sunset[np.where(day_length == 24)] = 99

    # Replace 99 and -99 with na in sunrise and sunset arrays
    if no_times_as_na is True:
        sunrise = np.where((sunrise == -99) | (sunrise == 99), np.nan, sunrise)
        sunset = np.where((sunset == -99) | (sunset == 99), np.nan, sunset)

    # # Check the presence of nan
    if (
        any(np.isnan(sunrise))
        or any(np.isnan(sunset))
        or any(np.isnan(day_length))
    ):
        warnings.warn('nan found in sunrise, sunset or day_length')

    # Return dictionary
    return collections.defaultdict(
        list, {'sunrise': sunrise, 'sunset': sunset, 'day_length': day_length}
    )

#### __Example: Calculate day length__

This function was originally developed in the ChillR R [package](https://cran.r-project.org/web/packages/chillR/chillR.pdf)

In [ ]:
day_length(latitude=45, day_of_year=100, no_times_as_na=False)

/tmp/ipykernel_27135/2141787043.py:9: UserWarning: Double check day_length function works for Australia
  warnings.warn('Double check day_length function works for Australia')


defaultdict(list,
            {'sunrise': array([5.4049135]),
             'sunset': array([18.5950865]),
             'day_length': array([13.190173])})

In [ ]:
day_length(
    latitude=50,
    day_of_year=np.array([78, 79, 80, 81, 82]),
    no_times_as_na=False,
)

/tmp/ipykernel_27135/2141787043.py:9: UserWarning: Double check day_length function works for Australia
  warnings.warn('Double check day_length function works for Australia')


defaultdict(list,
            {'sunrise': array([5.98160544, 5.95020317, 5.91880891, 5.88742546, 5.85605558]),
             'sunset': array([18.01839456, 18.04979683, 18.08119109, 18.11257454, 18.14394442]),
             'day_length': array([12.03678911, 12.09959365, 12.16238217, 12.22514908, 12.28788884])})